In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.2/213.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 4.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=4811d98b9fdd2167733b1d491bef055434bb187bbe6b3630ac950c8c19e6a890
  Stored in directory: /mnt/ceph/storage/data-tmp/2022/hj80pahi/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uni

# Defining the Model Training

## Dataset & Autoencoder

In [1]:
import torch
from configs.deepsvg.hierarchical_ordered import Config
from deepsvg import utils

pretrained_path = "./hierarchical_ordered.pth.tar"

device = torch.device("cuda:0"if torch.cuda.is_available() else "cpu")

cfg = Config()
vae_model = cfg.make_model().to(device)
utils.load_model(pretrained_path, vae_model)
vae_model.eval()

SVGTransformer(
  (encoder): Encoder(
    (embedding): SVGEmbedding(
      (command_embed): Embedding(7, 256)
      (arg_embed): Embedding(257, 64)
      (embed_fcn): Linear(in_features=704, out_features=256, bias=True)
      (pos_encoding): PositionalEncodingLUT(
        (dropout): Dropout(p=0.1, inplace=False)
        (pos_embed): Embedding(32, 256)
      )
    )
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayerImproved(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=F

In [2]:
import torch
from deepsvg.utils.utils import batchify
from deepsvg.difflib.tensor import SVGTensor
from deepsvg.svglib.svg import SVG

def encode(data, model):
    model_args = batchify((data[key] for key in cfg.model_args), device)
    with torch.no_grad():
        z = model(*model_args, encode_mode=True)
        return z.squeeze(dim=0).squeeze(dim=0)

def decode(z, model, do_display=True, return_svg=False, return_png=False):
    commands_y, args_y = model.greedy_sample(z=z)
    tensor_pred = SVGTensor.from_cmd_args(commands_y[0].cpu(), args_y[0].cpu())
    svg_path_sample = SVG.from_tensor(tensor_pred.data, viewbox=Bbox(256), allow_empty=True).normalize().split_paths().set_color("random")

    if return_svg:
        return svg_path_sample

    return svg_path_sample.draw(do_display=do_display, return_png=return_png)

In [3]:
from deepsvg.svgtensor_dataset import load_dataset
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

dataset = load_dataset(cfg) # the DeepSVG dataset as {'commands': [...], 'args': [...]}


def dataloader_with_transformed_dataset(batch_n: int, length: int = None):
    encoded_dataset_with_labels = []
    data_len = length if length else len(dataset)

    for i in range(data_len):
        xy = dataset.get(i, model_args=['commands', 'args', 'label'])
        label = xy.pop('label')
        encoded_dataset_with_labels.append([encode(xy, vae_model), label])

    #   encoded_dataset.append(encoded_svg[0][0])

    dataset_size = len(encoded_dataset_with_labels)
    batch_size = batch_n
    validation_split = .2
    shuffle_dataset = True
    random_seed= 42

    # Creating data indices for training and validation splits:

    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(encoded_dataset_with_labels, batch_size=batch_size, sampler=train_sampler, drop_last=True,)
    validation_loader = DataLoader(encoded_dataset_with_labels, batch_size=batch_size, sampler=valid_sampler, drop_last=True,)

    return train_loader, validation_loader

In [4]:
def num_classes(dataloader):
    all_classes = set()

    for x, y in dataloader:
          all_classes.update(set(y.numpy()))

    return len(all_classes)

In [18]:
train_dataloader, valid_dataloader = dataloader_with_transformed_dataset(batch_n=100, length=1000)

## Model

In [5]:
from dit.diffusion import create_diffusion
from svgfusion import DiT

def create_model(predict_xstart=True, dropout=0.1, n_classes=56, depth=28, learn_sigma=True, num_heads=16):

    model = DiT(class_dropout_prob=dropout, num_classes=n_classes, depth=depth, learn_sigma=learn_sigma, num_heads=num_heads)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model.to(device)
    diffusion = create_diffusion(timestep_respacing="", predict_xstart=predict_xstart)  # default: 1000 steps, linear noise schedule

    model.train()  # important! This enables embedding dropout for classifier-free guidance
    
    return model, diffusion

In [30]:
from dit.diffusion.gaussian_diffusion import LossType, ModelVarType, ModelMeanType

def mean_flat(tensor):
    """
    Take the mean over all non-batch dimensions.
    """
    return tensor.mean(dim=list(range(1, len(tensor.shape))))

def training_losses(diffusion, model, x_start, t, model_kwargs=None, noise=None):
        """
        Compute training losses for a single timestep.
        :param model: the model to evaluate loss on.
        :param x_start: the [N x C x ...] tensor of inputs.
        :param t: a batch of timestep indices.
        :param model_kwargs: if not None, a dict of extra keyword arguments to
            pass to the model. This can be used for conditioning.
        :param noise: if specified, the specific Gaussian noise to try to remove.
        :return: a dict with the key "loss" containing a tensor of shape [N].
                 Some mean or variance settings may also have other keys.
        """
        if model_kwargs is None:
            model_kwargs = {}
        if noise is None:
            noise = torch.randn_like(x_start)
        x_t = diffusion.q_sample(x_start, t, noise=noise)

        terms = {}

        if diffusion.loss_type == LossType.KL or diffusion.loss_type == LossType.RESCALED_KL:
            terms["loss"] = diffusion._vb_terms_bpd(
                model=model,
                x_start=x_start,
                x_t=x_t,
                t=t,
                clip_denoised=False,
                model_kwargs=model_kwargs,
            )["output"]
            if diffusion.loss_type == LossType.RESCALED_KL:
                terms["loss"] *= diffusion.num_timesteps
        elif diffusion.loss_type == LossType.MSE or diffusion.loss_type == LossType.RESCALED_MSE:
            model_output = model(x_t, t, **model_kwargs)

            if diffusion.model_var_type in [
                ModelVarType.LEARNED,
                ModelVarType.LEARNED_RANGE,
            ]:
                B, C = x_t.shape[:2]
                assert model_output.shape == (B, C * 2, *x_t.shape[2:])
                model_output, model_var_values = torch.split(model_output, C, dim=1)
                # Learn the variance using the variational bound, but don't let
                # it affect our mean prediction.
                frozen_out = torch.cat([model_output.detach(), model_var_values], dim=1)
                terms["vb"] = diffusion._vb_terms_bpd(
                    model=lambda *args, r=frozen_out: r,
                    x_start=x_start,
                    x_t=x_t,
                    t=t,
                    clip_denoised=False,
                )["output"]
                if diffusion.loss_type == LossType.RESCALED_MSE:
                    # Divide by 1000 for equivalence with initial implementation.
                    # Without a factor of 1/1000, the VB term hurts the MSE term.
                    terms["vb"] *= diffusion.num_timesteps / 1000.0

            target = {
                ModelMeanType.PREVIOUS_X: diffusion.q_posterior_mean_variance(
                    x_start=x_start, x_t=x_t, t=t
                )[0],
                ModelMeanType.START_X: x_start,
                ModelMeanType.EPSILON: noise,
            }[diffusion.model_mean_type]
            assert model_output.shape == target.shape == x_start.shape
            terms["mse"] = mean_flat((target - model_output) ** 2)
            if "vb" in terms:
                terms["loss"] = terms["mse"] + terms["vb"]
            else:
                terms["loss"] = terms["mse"]
        else:
            raise NotImplementedError(diffusion.loss_type)

        return terms

In [42]:
config = {
        'optimizer': 'adam',
        'predict_xstart': True,
        'learn_sigma': True,
        'use_schduler': True,
        'num_heads': 16,
        'depth': 28,
        'dropout': 0.1,
        'epochs': 1,
        'learning_rate': 0.001,
        'batch_size': 100,
}

model, diffusion = create_model(dropout=config['dropout'], predict_xstart=config['predict_xstart'],
                                    n_classes=num_classes(train_dataloader), depth=config['depth'], 
                                    learn_sigma=config['learn_sigma'], num_heads=config['num_heads'])
tmp = next(iter(train_dataloader))
t = torch.randint(0, diffusion.num_timesteps, (tmp[0].shape[0],), device=device)
model(tmp[0].to(device), t, tmp[1].to(device)).shape

torch.Size([100, 2, 256])

In [43]:
training_losses(diffusion, model, tmp[0].to(device), t, {'y': tmp[1].to(device)})["loss"].mean()

tensor(0.7577, device='cuda:0', grad_fn=<MeanBackward0>)

In [54]:
print(tmp[0].shape)
print(tmp[0].squeeze().unsqueeze(dim=0).shape)

torch.Size([100, 1, 256])
torch.Size([1, 100, 256])


## Training

In [6]:
import wandb
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

def train():
    config_defaults = {
            'optimizer': 'adam',
            'predict_xstart': True,
            'learn_sigma': True,
            'use_schduler': True,
            'num_heads': 16,
            'depth': 28,
            'dropout': 0.1,
            'epochs': 100,
            'learning_rate': 0.001,
            'batch_size': 100,
    }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_dataloader, valid_dataloader = dataloader_with_transformed_dataset(batch_n=config.batch_size, length=1000)

    model, diffusion = create_model(dropout=config.dropout, predict_xstart=config.predict_xstart,
                                    n_classes=num_classes(train_dataloader), depth=config.depth, 
                                    learn_sigma=config.learn_sigma, num_heads=config.num_heads)
    
    magical_number = 0.7128
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    optimizer = AdamW(model.parameters(), lr=config.learning_rate, weight_decay=0)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
    
    for epoch in range(config.epochs):
        avg_loss = 0
        for x, y in train_dataloader:
            x = x.to(device)
            y = y.to(device)
    
            x = x.squeeze().unsqueeze(dim=1)
            x = x / magical_number # mean of std's of latents
    
            model_kwargs = dict(y=y)
    
            t = torch.randint(0, diffusion.num_timesteps, (x.shape[0],), device=device)
    
            loss_dict = diffusion.training_losses(model, x, t, model_kwargs)
            loss = loss_dict["loss"].mean()
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()
            
            wandb.log({"batch_loss": loss.item()})     
        
        if config.use_schduler: scheduler.step(avg_loss / len(train_dataloader))
        wandb.log({"loss": avg_loss / len(train_dataloader), "epoch": epoch, 'learning_rate': optimizer.param_groups[0]['lr']}) 

In [60]:
train()
wandb.finish()

24
torch.Size([24, 1, 256])
torch.Size([24, 1, 256])
torch.Size([24, 1, 256])
(24, 2, 256)


AssertionError: 

# Defining the Sweep

## Config

In [7]:
import wandb

wandb.login()

wandb: Currently logged in as: hasanjbara. Use `wandb login --relogin` to force relogin


True

In [8]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'optimizer': {
            'values': ['adam']
        },
        'predict_xstart': {
            'value': True,
        },
        'learn_sigma':{
            'value': True, # [True, False]
        },
        'use_scheduler':{
            'values': [True, False],
        },
        'num_heads': {
            'values': [16, 32, 64, 128, 256]
        },
        'depth': {
            'distribution': 'int_uniform',
            'min': 28,
            'max': 100, 
        },
        'dropout': {
              'values': [0.3, 0.4, 0.5]
        },
        'epochs': {
            'value': 100
        },
        'learning_rate': {
            # a flat distribution between 0.01 and 0.0001
            'distribution': 'uniform',
            'min': 0.0001,
            'max': 0.01
        },
        'batch_size': {
            # integers between 32 and 256
            # with evenly-distributed logarithms 
            'distribution': 'q_log_uniform_values',
            'q': 8,
            'min': 16,
            'max': 128,
        }
    },
}

## Sweep

In [9]:
import wandb

sweep_id = wandb.sweep(sweep_config, project="svgfusion-sweep")

Create sweep with ID: 64ivnjiq
Sweep URL: https://wandb.ai/hasanjbara/svgfusion-sweep/sweeps/64ivnjiq


In [ ]:
import wandb

wandb.agent(sweep_id, train, count=100)

wandb: Agent Starting Run: f9i5202s with config:
wandb: 	batch_size: 72
wandb: 	depth: 29
wandb: 	dropout: 0.5
wandb: 	epochs: 100
wandb: 	learn_sigma: True
wandb: 	learning_rate: 0.00105478693528865
wandb: 	num_heads: 128
wandb: 	optimizer: adam
wandb: 	predict_xstart: True
wandb: 	use_scheduler: True
